In [1]:
import torch
import transformers
from tqdm import tqdm
import torch.nn as nn
import numpy as np
from sklearn.metrics import f1_score
import gc
import json
from time import time
import os
import albumentations as A
import cv2
from dataclasses import dataclass

In [2]:
from src.neural_nets import *
from src.utils import *
from src.train_utils import *
from transformers import AutoImageProcessor
import json
from torch.utils.data import DataLoader

In [3]:
# По конфигу начать обучение
with open(CONFIG_FILE_JSON, 'r', encoding='utf-8') as fd:
    json_obj = json.loads(fd.read())
learn_config = LearningConfig(**json_obj)

In [4]:
if learn_config.model_name == 'resnet':
    model = ResNet50Classifier(14)
    proc_f = AutoImageProcessor.from_pretrained(RESNET_MODEL_NAME)
    processor = lambda x: torch.tensor(proc_f(x)['pixel_values'][0])
    print("Выбран ResNet-based классификатор")

elif learn_config.model_name == 'mask2former':
    model = Mask2FormerClassifier(14)
    proc_f = AutoImageProcessor.from_pretrained(M2F_MODEL_NAME)
    processor = lambda x: torch.tensor(proc_f(x)['pixel_values'][0])
    print("Выбран Mask2Former-based классификатор")

elif learn_config.model_name == 'mycnn':
    model = MyCNNClassifier(14)
    processor = lambda x: TRANSFORM_MYCNN(x)
    print("Выбран самописный СNN-based классификатор")

else:
    print("ERROR: Invalid model name!")

Выбран самописный СNN-based классификатор


In [5]:
train_dataset = CustomCarDataset('train', TT_INFO, processor)
eval_dataset = CustomCarDataset('eval', TT_INFO, processor)

In [6]:
# Dataloader
train_dataloader = DataLoader(train_dataset, batch_size=learn_config.batch_size, 
                              shuffle=True, 
                              collate_fn=custom_collate)
eval_dataloader = DataLoader(eval_dataset, batch_size=learn_config.batch_size,
                              collate_fn=custom_collate)

In [7]:
x = model(next(iter(train_dataloader))['images'])

In [14]:
torch.argmax(x, dim=-1)

tensor([ 8,  8, 10,  6, 10,  8, 13,  2])

In [17]:
f1_score([0,1,2,3],[0,1,2,3], average='micro')

1.0

In [18]:
accuracy_score([0,1,2,3],[0,1,2,3])

1.0

In [11]:
l = next(iter(train_dataloader))['labels']

In [10]:
criterion = nn.CrossEntropyLoss()

In [13]:
criterion(x, l)

tensor(2.6381, grad_fn=<NllLossBackward0>)

In [8]:
x.shape

torch.Size([8, 14])

In [11]:
run(learn_config, model, train_dataloader, eval_dataloader)

Model parameters count:  199796
Init folder to save
Error: Директория существует
